# Exercício
## Ensembles e K-nearest neighbors
### Alunos (Nome e número usp):
- Fernanda Tostes Marana (4471070)
- Matheus Aparecido do Carmo Alves (9791114)
- Thais Bianchini (9791010)
 
---

Para esse exercício, vamos utilizar novamente o dataset [Breast Cancer Winsconsin](https://scikit-learn.org/stable/datasets/index.html#breast-cancer-dataset).



---

### Questão 1.

Carregue o dataset Breast Cancer do módulo `sklearn.datasets` e normalize os dados.

In [0]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

# 1. Loading breast cancer dataset
breast_cancer = datasets.load_breast_cancer()
X = breast_cancer['data']
y = breast_cancer['target']

# 2. Normalising the data
# - here we are standardising features by removing the mean and scaling to unit 
# variance, i.e.: z = (x-u)/s, where x is the data vector, u is the mean and
# s is the standard deviation 
scaler = StandardScaler()
X = scaler.fit_transform(X,y)

# 3. Presenting the normalised dataset
data = np.column_stack((X,y))
feature_names = list(breast_cancer['feature_names'])
feature_names.append('class')
breast_cancer_df = pd.DataFrame(data,columns=feature_names)
breast_cancer_df

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,class
0,1.097064,-2.073335,1.269934,0.984375,1.568466,3.283515,2.652874,2.532475,2.217515,2.255747,2.489734,-0.565265,2.833031,2.487578,-0.214002,1.316862,0.724026,0.660820,1.148757,0.907083,1.886690,-1.359293,2.303601,2.001237,1.307686,2.616665,2.109526,2.296076,2.750622,1.937015,0.0
1,1.829821,-0.353632,1.685955,1.908708,-0.826962,-0.487072,-0.023846,0.548144,0.001392,-0.868652,0.499255,-0.876244,0.263327,0.742402,-0.605351,-0.692926,-0.440780,0.260162,-0.805450,-0.099444,1.805927,-0.369203,1.535126,1.890489,-0.375612,-0.430444,-0.146749,1.087084,-0.243890,0.281190,0.0
2,1.579888,0.456187,1.566503,1.558884,0.942210,1.052926,1.363478,2.037231,0.939685,-0.398008,1.228676,-0.780083,0.850928,1.181336,-0.297005,0.814974,0.213076,1.424827,0.237036,0.293559,1.511870,-0.023974,1.347475,1.456285,0.527407,1.082932,0.854974,1.955000,1.152255,0.201391,0.0
3,-0.768909,0.253732,-0.592687,-0.764464,3.283553,3.402909,1.915897,1.451707,2.867383,4.910919,0.326373,-0.110409,0.286593,-0.288378,0.689702,2.744280,0.819518,1.115007,4.732680,2.047511,-0.281464,0.133984,-0.249939,-0.550021,3.394275,3.893397,1.989588,2.175786,6.046041,4.935010,0.0
4,1.750297,-1.151816,1.776573,1.826229,0.280372,0.539340,1.371011,1.428493,-0.009560,-0.562450,1.270543,-0.790244,1.273189,1.190357,1.483067,-0.048520,0.828471,1.144205,-0.361092,0.499328,1.298575,-1.466770,1.338539,1.220724,0.220556,-0.313395,0.613179,0.729259,-0.868353,-0.397100,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,2.110995,0.721473,2.060786,2.343856,1.041842,0.219060,1.947285,2.320965,-0.312589,-0.931027,2.782080,0.071025,2.379583,2.604187,1.086384,0.191805,0.666001,2.067178,-1.138416,0.167980,1.901185,0.117700,1.752563,2.015301,0.378365,-0.273318,0.664512,1.629151,-1.360158,-0.709091,0.0
565,1.704854,2.085134,1.615931,1.723842,0.102458,-0.017833,0.693043,1.263669,-0.217664,-1.058611,1.300499,2.260938,1.156857,1.291565,-0.424010,-0.069758,0.252202,0.808431,-0.189161,-0.490556,1.536720,2.047399,1.421940,1.494959,-0.691230,-0.394820,0.236573,0.733827,-0.531855,-0.973978,0.0
566,0.702284,2.045574,0.672676,0.577953,-0.840484,-0.038680,0.046588,0.105777,-0.809117,-0.895587,0.184892,-0.257371,0.276693,0.180698,-0.379342,0.661277,0.510827,0.612157,-0.891416,0.036727,0.561361,1.374854,0.579001,0.427906,-0.809587,0.350735,0.326767,0.414069,-1.104549,-0.318409,0.0
567,1.838341,2.336457,1.982524,1.735218,1.525767,3.272144,3.296944,2.658866,2.137194,1.043695,1.157935,0.686088,1.438530,1.009503,-0.173000,2.017716,1.302285,0.785721,0.326634,0.904057,1.961239,2.237926,2.303601,1.653171,1.430427,3.904848,3.197605,2.289985,1.919083,2.219635,0.0




---

### Questão 2.

Aqui definiremos funções que serão utilizadas para simplificar as operações que faremos posteriormente. Dessa forma, implemente as funções abaixo:

*   A função `get_mean_accuracy(model, X, y)` recebe um modelo `model` e um conjunto de atributos `X` e labels `y`. Ela deve calcular a acurácia do modelo utilizando 10-fold cross-validation estratificado e retornar a acurácia média dos 10 folds
*   A função `evaluate_models(models, X, y)` recebe um conjunto de modelos definidos por um dicionário e exibe, para cada modelo, seu nome seguido da sua acurácia (calculada com a função `get_mean_accuracy`). Um exemplo de saída para o dicionário `exemplo` abaixo seria:
> A acurácia do modelo "Knn (n_neighbors = 5)" é 85.00%
>
> A acurácia do modelo "DT (gini)" é 80.00%

*   Finalmente, implemente a função `create_name_list(models)`. Essa função deve receber um dicionário e retornar uma lista contendo, para cada elemento do dicionário, uma tupla (chave, valor). Para o dicionário `exemplo`, essa função retornaria:
> `[ ('Knn (n_neighbors = 5)', KNeighborsClassifier(n_neighbors=5)), ('DT (gini)', DecisionTreeClassifier(criterion="gini"))]`


```
exemplo = {
    'Knn (n_neighbors = 5)': KNeighborsClassifier(n_neighbors=5),
    'DT (gini)': DecisionTreeClassifier(criterion="gini"),
}
```



In [0]:
from numpy import mean
from sklearn.model_selection import cross_validate

def get_mean_accuracy(model, X, y):
  # Applying the stratified 10-fold cross-valitation
  metrics = cross_validate(model, X, y, cv=10)

  # Returning the mean accuracy score
  return(mean(metrics['test_score'])) 

def evaluate_models(models, X, y):
  for (model_name,model) in models:
    print('A acurácia do modelo \"'+str(model_name)+'\" é '+\
          str(np.round(100*get_mean_accuracy(model,X,y),4))+'%')

def create_name_list(models):
  list1 = [(k, v) for k, v in models.items()]
  return (list1)



---

### Questão 3.

Defina modelos de SVM (`sklearn.svm.SVC`) e MLP (`sklearn.neural_network.MLPClassifier`) para servir de *baseline* de comparação para modelos futuros. Para isso, teste ao menos 3 configurações de SVM e 3 configurações de MLP. Sua baseline será a melhor acurácia (utilizando 10-fold cross-validation estratificado) entre essas configurações. Utilize as funções definidas na Questão 2 quando necessário.

In [0]:
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

experiments = { 
    "SVM Linear" : SVC(kernel='poly',degree=1,probability=True),
    "SVM 3-Poly" : SVC(kernel='poly',degree=3,probability=True),
    "SVM 9-Poly" : SVC(kernel='poly',degree=9,probability=True),
    "MLP 3-HL" : MLPClassifier(hidden_layer_sizes=(30,30,30,),solver='lbfgs'),
    "MLP 6-HL" : MLPClassifier(hidden_layer_sizes=(30,30,30,30,30,30,),solver='lbfgs'),  
    "MLP 9-HL" : MLPClassifier(hidden_layer_sizes=(30,30,30,30,30,30,30,30,30,),solver='lbfgs'),
}
list_ready = create_name_list(experiments)
evaluate_models(list_ready,X,y)

A acurácia do modelo "SVM Linear" é 97.713%
A acurácia do modelo "SVM 3-Poly" é 90.3383%
A acurácia do modelo "SVM 9-Poly" é 79.2638%
A acurácia do modelo "MLP 3-HL" é 97.3622%
A acurácia do modelo "MLP 6-HL" é 95.4323%
A acurácia do modelo "MLP 9-HL" é 90.1598%




---

### Questão 4.

Agora defina dois dicionários de classificadores. Um dicionário deve conter apenas classificadores KNN (`sklearn.neighbors.KNeighborsClassifier`), enquanto o outro deve conter apenas classificadores DT (`sklearn.tree.DecisionTreeClassifier`). Crie ao menos 3 configurações diferentes para cada tipo de classificador. Depois calcule e exiba a acurácia de cada configuração criada utilizando 10-fold cross-validation estratificado.

In [0]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

experiments_knn = { 
    "KNN 10-NN" : KNeighborsClassifier(n_neighbors=10),
    "KNN 25-NN" : KNeighborsClassifier(n_neighbors=25),
    "KNN 50-NN" : KNeighborsClassifier(n_neighbors=50),
}
experiments_dt = { 
    "DTC 10-MD (gini)" : DecisionTreeClassifier(criterion='gini',max_depth=10),
    "DTC 10-MD (entropy)" : DecisionTreeClassifier(criterion='entropy',max_depth=10),
    "DTC 25-MD (gini)" : DecisionTreeClassifier(criterion='gini',max_depth=25),
    "DTC 25-MD (entropy)" : DecisionTreeClassifier(criterion='entropy',max_depth=25),
    "DTC 50-MD (gini)" : DecisionTreeClassifier(criterion='gini',max_depth=50),
    "DTC 50-MD (entropy)" : DecisionTreeClassifier(criterion='entropy',max_depth=50),
}

# Testing knn models
list_ready = create_name_list(experiments_knn)
evaluate_models(list_ready,X,y)

# Testing dt models
list_ready = create_name_list(experiments_dt)
evaluate_models(list_ready,X,y)

A acurácia do modelo "KNN 10-NN" é 97.0113%
A acurácia do modelo "KNN 25-NN" é 95.6078%
A acurácia do modelo "KNN 50-NN" é 95.4323%
A acurácia do modelo "DTC 10-MD (gini)" é 90.8741%
A acurácia do modelo "DTC 10-MD (entropy)" é 92.7882%
A acurácia do modelo "DTC 25-MD (gini)" é 91.2187%
A acurácia do modelo "DTC 25-MD (entropy)" é 92.7851%
A acurácia do modelo "DTC 50-MD (gini)" é 91.3972%
A acurácia do modelo "DTC 50-MD (entropy)" é 92.9637%




---

### Questão 5.

Agora você deve definir um dicionário contendo diferentes configurações de ensembles a serem testados. Utilize as classes `StackingClassifier`, `AdaBoostClassifier` e `VotingClassifier` do módulo `sklearn.ensemble` (fique a vontade para escolher outros tipos de ensembles desse mesmo módulo). Crie pelo menos 5 configurações diferentes tomando como estimadores os modelos definidos na Questão 4. Calcule a acurácia de cada configuração de ensemble. 

Obs.: Lembre-se de utilizar as funções definidas anteriormente

In [0]:
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier

estimators = [('SVM Linear',experiments['SVM Linear']),\
              ('MLP 3-HL',experiments['MLP 3-HL']),\
              ('KNN 10-NN',experiments_knn['KNN 10-NN']),\
              ('DTC 10-MD (entropy)',experiments_dt['DTC 10-MD (entropy)'])]

experiments_ens = { 
    "SC PP" : StackingClassifier(estimators,cv=10,stack_method='predict_proba'),
    "SC P " : StackingClassifier(estimators,cv=10,stack_method='predict'),
    "ABC 1.0-LR" : AdaBoostClassifier( n_estimators=50, learning_rate=1.0),
    "ABC 0.7-LR" : AdaBoostClassifier( n_estimators=50, learning_rate=0.7),
    "ABC 0.5-LR" : AdaBoostClassifier( n_estimators=50, learning_rate=0.5),
    "VC Soft" : VotingClassifier(estimators,voting='soft'),
    "VC Hard" : VotingClassifier(estimators,voting='hard'),
}

list_ready = create_name_list(experiments_ens)
evaluate_models(list_ready,X,y)

A acurácia do modelo "SC PP" é 98.4148%
A acurácia do modelo "SC P " é 97.5376%
A acurácia do modelo "ABC 1.0-LR" é 96.131%
A acurácia do modelo "ABC 0.7-LR" é 96.6604%
A acurácia do modelo "ABC 0.5-LR" é 95.9586%
A acurácia do modelo "VC Soft" é 97.5376%
A acurácia do modelo "VC Hard" é 97.005%




---

### Questão 6. 

Discuta as acurácias obtidas nas Questões 3, 4 e 5. Houve algum ganho ao utilizar Ensembles? Em qual situação houve maior ganho?

**RESPOSTA:**

Resguardando as acurácias obtidas nas Questões 3, 4 e 5, temos que as melhores classificações de cada modelo foram:

* **SVM Linear :** 97.713%
* **MLP 3-HL :** 95.0815%
* **KNN 10-NN :** 97.0113%
* **DTC 10-MD (entropy):** 93.6779%

Estes melhores resultados guiaram a decisão do conjunto de estimadores a serem acoplados nos Ensembles.

Ao se utilizar estes métodos sozinhos, como classificadores (considerando as arquiteturas definidas neste exercício), alcançou-se a melhor acurácia com a aplicação da SVM Linear sobre o problema (97.713% de acurácia).

A partir da aplicação de Ensembles, foi possível aprimorar esse resultado significativamente, alcançando uma acúracia maior que 98% na utilização do método de Stacking como Ensemble e classificador final (arquitetura "SC PP"), podendo variar um pouco devido a natureza de aleatorização do algoritmo.
Esta melhora era esperada, dada a natureza e funcionamento de Ensembles.

De maneira direta, todos os Ensambles garantiram uma melhora na classificação final para algum dos classificadores utilizados. O que mais se aprimorou foi o modelo DTC 10-MD que antes apresentava um menor taxa de acurácia em seus resultados. Adicionalmente, a situação que garantiu o maior ganho foi a utilização da arquitetura "SC PP" (como comentado anteriormente).

Desta forma, pode-se observar que existe ganho na aplicação de Ensembles, sendo necessário somente estudá-los e modelá-los para melhorar o resultado para o problema como um todo.
